In [1]:
import pandas as pd

In [2]:
dfglobal = pd.read_csv("Classification/all_files_globalv1.csv")
dfza = pd.read_csv("Classification/all_files_ZAv1.csv")
dfskr = pd.read_csv("Classification/all_files_skrv1.csv")

dfskr=dfskr.drop_duplicates()

In [3]:
artistsbystreams = dfskr.groupby(["main_artist"])["streams"].sum()
artistsbystreamssorted = pd.DataFrame(artistsbystreams.sort_values(ascending=False))
artistsbystreamssorted=artistsbystreamssorted.reset_index()

In [4]:
groupname=["bts", "blackpink"]
groups = [["v", "jimin", "jin", 'jung kook', 'agust d', 'j-hope', 'rm'], ["jennie", 'jisoo', 'lisa', 'ros']]
groupsuppercase = [["V", "Jimin", "Jin", 'Jung Kook', 'Agust D', 'j-hope', 'RM'], ["JENNIE", 'JISOO', 'LISA', 'ROS']]
dfskr['lower_main_artist'] = dfskr['main_artist'].apply(lambda x :x.lower())
idx = 1

rslt_df = dfskr[dfskr['lower_main_artist'].isin(groups[idx])]
rslt_df['main_artist'].unique().tolist()

rslt_df = rslt_df[rslt_df['main_artist'] != 'LiSA']

In [5]:
rslt_df["main_artist"].unique().tolist()

['JENNIE', 'JISOO', 'LISA', 'ROS']

### Number of songs

In [7]:
rslt_df[["main_artist","track_name"]].drop_duplicates()

numsongsdf = pd.DataFrame(rslt_df.groupby(["main_artist","track_name"])["track_name"].count())
numsongsdf.columns = ["appearances"]
numsongsdf = numsongsdf.reset_index()
numsongscount = pd.DataFrame(numsongsdf.groupby(["main_artist"])["track_name"].count())
numsongscount.columns = ["num_songs"]
numsongscountdf = numsongscount.reset_index()

### Number of appearances

In [9]:
numappearancescount = pd.DataFrame(numsongsdf.groupby(["main_artist"])["appearances"].sum())
numappearancescountdf = numappearancescount.reset_index()

### Streams per artist

In [11]:
numstreamsdf = pd.DataFrame(rslt_df.groupby(["main_artist"])["streams"].sum())
numstreamsdf.columns = ["streams_sum"]
numstreamsdf = numstreamsdf.reset_index()

### Years on the chart

In [13]:
yrsonchartdf = pd.DataFrame(rslt_df.groupby(["main_artist"])["Year"].nunique())
yrsonchartdf.columns = ["yrsonchart"]
yrsonchartdf = yrsonchartdf.reset_index()

### Collaborators

In [15]:
thegrp = groupsuppercase[idx]
thecollaboratorsdict = {}
for aval in thegrp:
    rslt_dfoneartist = pd.DataFrame(rslt_df[rslt_df["main_artist"]==aval]["artist_names"])
    rslt_dfoneartist = rslt_dfoneartist.drop_duplicates()

    thecolls = []
    for theval in rslt_dfoneartist["artist_names"].unique().tolist():
        splitvals = theval.split(",")
        thecollaborators = list(set(splitvals) - set([aval]))

        for thec in thecollaborators:
            thec = thec.strip()
            thecolls.append(thec)
    thecolls=list(set(thecolls))
    thecollaboratorsdict[aval] = [thecolls, len(thecolls)]
    
finnumcollabs = []
for aval in thegrp:
    finnumcollabs.append([aval, thecollaboratorsdict[aval][1]])
numcollabsdf = pd.DataFrame(finnumcollabs, columns=["main_artist", "num_collaborators"])

In [16]:
# thecollaboratorsdict["Jung Kook"]

### FINAL DATAFRAME

In [18]:
numappandsongs = numappearancescountdf.merge(numsongscountdf, left_on=["main_artist"], right_on=["main_artist"], how="inner")
numappstreamssongs = numstreamsdf.merge(numappandsongs, left_on=["main_artist"], right_on=["main_artist"], how="inner")
numappstreamssongsyrs = numappstreamssongs.merge(yrsonchartdf, left_on=["main_artist"], right_on=["main_artist"], how="inner")
numappstreamssongsyrscollabs = numappstreamssongsyrs.merge(numcollabsdf, left_on=["main_artist"], right_on=["main_artist"], how="left")
numappstreamssongsyrscollabs.to_csv("Classification/numappstreamssongsyrs" + groupname[idx] + ".csv", index=False)